In [4]:
from transformers import pipeline
model_sentiment = pipeline("sentiment-analysis", model="mrm8488/deberta-v3-small-finetuned-sst2",device=0)
#model_sentiment = pipeline("sentiment-analysis",device=0)

c:\ProgramData\Anaconda3\envs\trans\lib\site-packages\transformers\convert_slow_tokenizer.py:446: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(


In [5]:
import pandas as pd
import numpy as np
df = pd.read_csv("chat_dataset.csv")

In [6]:
ctext = "message"
sent_col = "sentiment_m"

In [8]:
df[sent_col] = "NEGATIVE"
df[sent_col+"_VALUE"] = 0
i = 0
sent = []
s_val = []
for s in df[ctext].to_list():
    #print(s)
    i+=1
    if i%100 ==0:
        print(i,end=" ")
    if len(s)>1:
        if(len(s)>2000):
            s=s[:2000]
        result = model_sentiment(s)[0]
        if result['label'].upper() == "NEGATIVE":
            sent.append("negative")
            s_val.append(-result['score'])
        else:
            sent.append("positive")
            s_val.append(result['score'])
    else:
        sent.append(0)
        s_val.append(0)
df[sent_col] = sent
df[sent_col+"_VALUE"] = s_val

c:\ProgramData\Anaconda3\envs\trans\lib\site-packages\transformers\pipelines\base.py:1045: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(


100 200 300 400 500 

In [9]:
df

,message,sentiment,sentiment_m,sentiment_m_VALUE
0,I really enjoyed the movie,positive,positive,0.999548
1,The food was terrible,negative,negative,-0.997103
2,I'm not sure how I feel about this,neutral,negative,-0.993515
3,The service was excellent,positive,positive,0.999526
4,I had a bad experience,negative,negative,-0.996874
...,...,...,...,...
579,I have to cancel my vacation plans because I c...,negative,negative,-0.988882
580,My computer crashed and I lost all my importan...,negative,negative,-0.995366
581,I got into a car accident and my car is totale...,negative,negative,-0.994818
582,I have a cold and can't stop coughing. it's re...,negative,negative,-0.989181


In [10]:
import openai
import tiktoken
encoding = tiktoken.get_encoding("gpt2")
with open('c:\\data\\openAI-key-g.txt', 'r') as file:
    key = file.readline()
openai.api_key = key

In [58]:
import json
def batch_gpt(prompt,target_texts, batch_size):
    l = len(target_texts)
    size = int(l / batch_size) + 1
    text_list = np.array_split(target_texts, size)
    print(f"Total records {l}, number of chunks = {size}")
    rez_keys = []
    rez_vals = []
    rez = []
    for i, texts in enumerate(text_list):
        text = "\n".join(texts)
        p = prompt + text
        # print(p)
        response = openai.ChatCompletion.create(
            model="gpt-3.5-turbo",
            messages=[{"role": "user", "content": p}]
        )
        r = response["choices"][0]["message"]["content"]
        # print(r)
        rez.append(r)
        try: 
            dictData= json.loads(r)
            keys = list(dictData.keys())
            values = list(dictData.values())
            rez_keys += keys
            rez_vals += values
        except:
            print("error parsing"+r)
        print(f"i={i + 1}: shape:{len(rez_keys)}")
    return rez_keys, rez_vals, rez

In [23]:
texts = df["message"].to_list()
prompt = 'Detect the sentiment of the texts and return the result in JSON without any other explanations, '+\
         'in the format {"text":"sentiment"\n"text":"sentiment"\n"text":"sentiment"}:\n' 
rez_keys, rez_vals, rez = batch_gpt(prompt, texts, 30)

Total records 584, number of chunks = 20
Detect the sentiment of the texts and return the result in JSON without any other explanations, in the format {"text":"sentiment"
"text":"sentiment"
"text":"sentiment"}:
I really enjoyed the movie
The food was terrible
I'm not sure how I feel about this
The service was excellent
I had a bad experience
This is a great product
I wouldn't recommend this
The price is a bit high
The weather is perfect today
The traffic is horrible
I'm feeling neutral about this
The concert was amazing
The customer service was terrible
This book is a masterpiece
I regret buying this product
The view from here is breathtaking
I don't know what to think about this
The internet speed is slow
The museum was fascinating
The noise level is too high
I don't have an opinion on this
The hotel room was spacious and clean
The service at this restaurant was awful
This software is very user-friendly
The quality of the product is poor
I don't really care about this
The beach was cr

In [25]:
df["gpt_text"] = rez_keys
df["gpt_sentiment"] = rez_vals

In [26]:
df.to_csv("chat_gpt.csv")

In [42]:
t = "I just got a raise and my salary is now 11000"
model_sentiment(t)

c:\ProgramData\Anaconda3\envs\trans\lib\site-packages\transformers\pipelines\base.py:1045: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(


[{'label': 'negative', 'score': 0.8940573334693909}]

In [44]:
tt = ["relatively slow and inefficient",
"Work formats change all the time and a clear work strategy is not matured",
"Too little manpower and resources for operations",
"Operations need to be simpilfied and to cut the red tapes",
"One cannot speak of good, one tries to slowly but always go there.",
"Do you even have to ask this question? On all previous surveys, the processes that slow down our work has been a major, consistent pain point. The opposite is true in many cases We cling to barriers and cumbersome processes instead of trying something new..",
"There is still a lot of personal work and manual work.",
"very sluggish and slow",
"Unfortunately, it has become necessary in many places to fill many documents, this takes a lot of time.",
"just a nice story they say in call. on the ground is a different situation and leadership afraid to remove individuls causing problems",
"This does happen sometimes, but unfortunately not often enough.",
"A change in IT support would be necessary. This wastes immense resources.",
]
model_sentiment(tt)

c:\ProgramData\Anaconda3\envs\trans\lib\site-packages\transformers\pipelines\base.py:1045: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(


[{'label': 'negative', 'score': 0.9973506927490234},
 {'label': 'negative', 'score': 0.9901617765426636},
 {'label': 'negative', 'score': 0.9963974356651306},
 {'label': 'negative', 'score': 0.9874020218849182},
 {'label': 'negative', 'score': 0.8754264116287231},
 {'label': 'negative', 'score': 0.9892222881317139},
 {'label': 'positive', 'score': 0.9365252256393433},
 {'label': 'negative', 'score': 0.997658371925354},
 {'label': 'negative', 'score': 0.9883885979652405},
 {'label': 'positive', 'score': 0.9972139000892639},
 {'label': 'negative', 'score': 0.8816083073616028},
 {'label': 'negative', 'score': 0.9937794804573059}]

In [45]:
prompt = 'Detect the sentiment of the texts and return the result in JSON without any other explanations, '+\
         'in the format {"text":"sentiment"\n"text":"sentiment"\n"text":"sentiment"}:\n' 
rez_keys, rez_vals, rez = batch_gpt(prompt, tt, 30)

Total records 12, number of chunks = 1
Detect the sentiment of the texts and return the result in JSON without any other explanations, in the format {"text":"sentiment"
"text":"sentiment"
"text":"sentiment"}:
relatively slow and inefficient
Work formats change all the time and a clear work strategy is not matured
Too little manpower and resources for operations
Operations need to be simpilfied and to cut the red tapes
One cannot speak of good, one tries to slowly but always go there.
Do you even have to ask this question? On all previous surveys, the processes that slow down our work has been a major, consistent pain point. The opposite is true in many cases We cling to barriers and cumbersome processes instead of trying something new..
There is still a lot of personal work and manual work.
very sluggish and slow
Unfortunately, it has become necessary in many places to fill many documents, this takes a lot of time.
just a nice story they say in call. on the ground is a different situat

In [48]:
print(rez[0])

{
"relatively slow and inefficient":"negative",
"Work formats change all the time and a clear work strategy is not matured":"negative",
"Too little manpower and resources for operations":"negative",
"Operations need to be simpilfied and to cut the red tapes":"negative",
"One cannot speak of good, one tries to slowly but always go there.":"neutral",
"Do you even have to ask this question? On all previous surveys, the processes that slow down our work has been a major, consistent pain point. The opposite is true in many cases We cling to barriers and cumbersome processes instead of trying something new..":"negative",
"There is still a lot of personal work and manual work.":"negative",
"very sluggish and slow":"negative",
"Unfortunately, it has become necessary in many places to fill many documents, this takes a lot of time.":"negative",
"just a nice story they say in call. on the ground is a different situation and leadership afraid to remove individuls causing problems":"negative",
"Thi

In [49]:
ttt = [
"The Rock is destined to be the 21st Century's new ``Conan'' and that he's going to make a splash even greater than Arnold Schwarzenegger, Jean-Claud Van Damme or Steven Segal.",
"The gorgeously elaborate continuation of ``The Lord of the Rings'' trilogy is so huge that a column of words cannot adequately describe co-writer/director Peter Jackson's expanded vision of J.R.R. Tolkien's Middle-earth.",
"Effective but too-tepid biopic",
"If you sometimes like to go to the movies to have fun, Wasabi is a good place to start.",
"Emerges as something rare, an issue movie that's so honest and keenly observed that it doesn't feel like one.",
"The film provides some great insight into the neurotic mindset of all comics -- even those who have reached the absolute top of the game.",
"Offers that rare combination of entertainment and education.",
"Perhaps no picture ever made has more literally showed that the road to hell is paved with good intentions.",
"Steers turns in a snappy screenplay that curls at the edges; it's so clever you want to hate it. But he somehow pulls it off.",
"Take Care of My Cat offers a refreshingly different slice of Asian cinema.",
"This is a film well worth seeing, talking and singing heads and all.",
"What really surprises about Wisegirls is its low-key quality and genuine tenderness.",
"(Wendigo is) why we go to the cinema: to be fed through the eye, the heart, the mind.",
"One of the greatest family-oriented, fantasy-adventure movies ever.",
"Ultimately, it ponders the reasons we need stories so much.",
"An utterly compelling 'who wrote it' in which the reputation of the most famous author who ever lived comes into question.",
"Illuminating if overly talky documentary.",
"A masterpiece four years in the making.",
"The movie's ripe, enrapturing beauty will tempt those willing to probe its inscrutable mysteries.",
"Offers a breath of the fresh air of true sophistication.",
"A thoughtful, provocative, insistently humanizing film.",
]

In [50]:
model_sentiment(ttt)

c:\ProgramData\Anaconda3\envs\trans\lib\site-packages\transformers\pipelines\base.py:1045: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(


[{'label': 'positive', 'score': 0.998087465763092},
 {'label': 'positive', 'score': 0.9994515776634216},
 {'label': 'negative', 'score': 0.97150719165802},
 {'label': 'positive', 'score': 0.9987322688102722},
 {'label': 'positive', 'score': 0.9978367686271667},
 {'label': 'positive', 'score': 0.9987505674362183},
 {'label': 'positive', 'score': 0.9992008805274963},
 {'label': 'negative', 'score': 0.6747934222221375},
 {'label': 'positive', 'score': 0.986312210559845},
 {'label': 'positive', 'score': 0.9993844032287598},
 {'label': 'positive', 'score': 0.9994099140167236},
 {'label': 'positive', 'score': 0.9966758489608765},
 {'label': 'positive', 'score': 0.9983502626419067},
 {'label': 'positive', 'score': 0.9994631409645081},
 {'label': 'positive', 'score': 0.9969528913497925},
 {'label': 'positive', 'score': 0.9990668892860413},
 {'label': 'positive', 'score': 0.9978710412979126},
 {'label': 'positive', 'score': 0.998703122138977},
 {'label': 'positive', 'score': 0.998988687992096},

In [57]:
rez_vals

['Positive',
 'Positive',
 'Neutral',
 'Positive',
 'Positive',
 'Positive',
 'Positive',
 'Negative',
 'Positive',
 'Positive',
 'Positive',
 'Positive',
 'Positive',
 'Positive',
 'Positive',
 'Positive',
 'Neutral',
 'Positive',
 'Positive',
 'Positive',
 'Positive']

In [59]:
prompt = 'Detect the sentiment of the texts and return the result in JSON without any other explanations, '+\
         'in the format {"text":"sentiment"\n"text":"sentiment"\n"text":"sentiment"}:\n' 
rez_keys, rez_vals, rez = batch_gpt(prompt, ttt, 30)
print(rez[0])

Total records 21, number of chunks = 1
i=1: shape:21
{"The Rock is destined to be the 21st Century's new ``Conan'' and that he's going to make a splash even greater than Arnold Schwarzenegger, Jean-Claud Van Damme or Steven Segal.": "Positive", "The gorgeously elaborate continuation of ``The Lord of the Rings'' trilogy is so huge that a column of words cannot adequately describe co-writer/director Peter Jackson's expanded vision of J.R.R. Tolkien's Middle-earth.": "Positive", "Effective but too-tepid biopic": "Neutral", "If you sometimes like to go to the movies to have fun, Wasabi is a good place to start.": "Positive", "Emerges as something rare, an issue movie that's so honest and keenly observed that it doesn't feel like one.": "Positive", "The film provides some great insight into the neurotic mindset of all comics -- even those who have reached the absolute top of the game.": "Positive", "Offers that rare combination of entertainment and education.": "Positive", "Perhaps no pictu

In [60]:
rez[0]

'{"The Rock is destined to be the 21st Century\'s new ``Conan\'\' and that he\'s going to make a splash even greater than Arnold Schwarzenegger, Jean-Claud Van Damme or Steven Segal.": "Positive", "The gorgeously elaborate continuation of ``The Lord of the Rings\'\' trilogy is so huge that a column of words cannot adequately describe co-writer/director Peter Jackson\'s expanded vision of J.R.R. Tolkien\'s Middle-earth.": "Positive", "Effective but too-tepid biopic": "Neutral", "If you sometimes like to go to the movies to have fun, Wasabi is a good place to start.": "Positive", "Emerges as something rare, an issue movie that\'s so honest and keenly observed that it doesn\'t feel like one.": "Positive", "The film provides some great insight into the neurotic mindset of all comics -- even those who have reached the absolute top of the game.": "Positive", "Offers that rare combination of entertainment and education.": "Positive", "Perhaps no picture ever made has more literally showed tha